In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [ ]:
# To Avoid GPU errors
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#### Importing Data from MNIST

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

### What we want to do
1. Save and Load Model Weights
2. Save and Load Entire Model (**Serialization**)
    1. Save Weights
    2. Save Model Architecture
    3. Training Configuration (model.compile())
    4. Optimizer and State
    
We will create three models with the three methods, Sequential, Functional and Subclassing

In [ ]:
# Sequential API
model1 = keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(10,activation='softmax')
])

In [ ]:
# Functional API
inputs = keras.Input(784)
x = layers.Dense(64, activation='relu')(inputs)
outputs = layers.Dense(10,activation='softmax')(x)
model2 = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
#subclassing
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = layers.Dense(64,activation='relu')
        self.dense2 = layers.Dense(10,activation='softmax')
    
    def call(self, input_tensor):
        x = tf.nn.relu(self.dense1(input_tensor))
        return self.dense2(x)

model3 = MyModel()

In [ ]:
model = model1
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

In [ ]:
#save weights
model.save_weights('saved_model/',save_format='h5')

In [ ]:
#load weights
model = model1
model.load_weights('saved_model/')
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

In [ ]:
#save entire model
model.save('complete_saved_models/')

In [ ]:
# now, to load, we can compile it as below
model = keras.models.load_model('complete_saved_models/')

#can directly train and fit and eval, as model is already compiled
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)